# Algorithme du gradient conjugué

In [ ]:
using LinearAlgebra
using Optim

Résolvez
$$
\min_x f(x) = \frac{1}{2} x^T A x + b^T x + a
$$
où $A \succ 0$. En posant $\nabla f(x) = 0$, c'est équivalent à résoudre le système linéaire $Ax = -b$.

Construisons la fonction quadratique associée au programme précédent.

In [ ]:
f = x -> 0.5*dot(x,A*x)+dot(b,x)

## Un exemple simple

Adapté de https://www.rose-hulman.edu/~bryan/lottamath/congrad.pdf

Soit
$$
A =
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
$$
Considérons la fonction à minimiser
$$
f(x) = \frac{1}{2} x^TAx,
$$
et supposons que nous avons déjà calculé
\begin{align*}
d_0 &= (1, 0, 0)\\
d_1 & = (1, −3, 0)\\
d_2 &= (−2, 6, −5).
\end{align*}

Vérifions que $d_0$, $d_1$ et $d_2$ sont $A$-conjugés.

In [ ]:
A = [ 3.0 1 0 ; 1 2 2 ; 0 2 4]
d0 = [ 1.0 0 0 ]'
d1 = [ 1.0 -3.0 0.0 ]'
d2 = [ -2.0 6.0 -5.0]'

println("$(dot(d0, A*d1)) $(dot(d0, A*d2)) $(dot(d1, A*d2))")

Vérifions que les valeurs propres de $A$ sont positif, et donc que le problème est convexe.

In [ ]:
eigen(A)

Prenons comme solution initiale $x_0 = (1, 2, 3)$. Calculons $x_1$, $x_2$ et $x_3$ en utilisant l'algorithme du gradient conjugué. $x_3$ est-il optimal?

Nous avons
$$
\nabla f(x) = Ax
$$

Nous définissons la fonction objectif comme suit.

In [ ]:
f = x -> dot(x,A*x)

Nous devons calculer $\alpha_k$, $k = 0,1,2$, en résolvant
$$
\min_{\alpha} f(x_k + \alpha d_k)
$$

Afin d'obtenir $\alpha_0$, nous devons minimiser
\begin{align*}
f(x_0 + \alpha d_0) &= \frac{1}{2}
\left(\begin{pmatrix} 1 & 2 & 3\end{pmatrix} + \alpha \begin{pmatrix} 1 & 0 & 0\end{pmatrix} \right)
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\left(\begin{pmatrix} 1 \\ 2 \\ 3 \end{pmatrix} + \alpha \begin{pmatrix} 1 \\0 \\0 \end{pmatrix} \right)
\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix}
3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4
\end{pmatrix}
\begin{pmatrix} 1 + \alpha \\ 2 \\ 3 \end{pmatrix}\\
& = \frac{1}{2}\begin{pmatrix} 1 + \alpha & 2 & 3 \end{pmatrix}
\begin{pmatrix} 5+3\alpha \\ 11+\alpha \\ 16 \end{pmatrix}\\
& = \frac{1}{2}
((1 + \alpha)(5+3\alpha) + 22+2\alpha + 48 ) \\
& = \frac{1}{2}
( 3\alpha^2 + 8\alpha + 5 + 70 + 2\alpha ) \\
& = \frac{3}{2}\alpha^2 + 5\alpha+\frac{75}{2}
\end{align*}
par rapport à $\alpha$.

Nous pouvons l'obtenir en cherchant le zéro de la dérivée par rapport à $\alpha$, c'est-à-dire
$$
\frac{d}{d\alpha} f(x+\alpha d) = 0,
$$
ou

$$
d^T \nabla f(x+\alpha d) = 0
$$

Dès lors, nous devons avoir

$$
3\alpha + 5 = 0
$$
Ainsi,
$$
\alpha_{0} = -\frac{5}{3}
$$
$$
x_1 = x_0 - \frac{5}{3} d_0 = \begin{pmatrix} -\frac{2}{3} \\ 2 \\ 3  \end{pmatrix}
$$

Nous pouvons aussi directement calculer $\alpha_0$ comme
$$
\alpha_0 = - \frac{d_0^T\nabla f(x_0)}{d_0^TAd_0}
$$

In [ ]:
x0 = [1 ; 2 ; 3.0]
∇f = x -> A*x
d0 = [1 ; 0 ; 0]
α0 = -dot(d0,∇f(x0))/dot(d0,A*d0)

Calculons le nouvel itéré.

In [ ]:
x1 = x0+α0*d0

Une recherche linéaire à partir de $x_1$ dans la direction $d_1$ exige de minimiser
\begin{align*}
f(x_1 + \alpha d_1) & = \left(\begin{pmatrix} -\frac{2}{3} & 2 & 3 \end{pmatrix} + \alpha_1\begin{pmatrix} 1 & -3 & 0 \end{pmatrix} \right)\begin{pmatrix} 3 & 1 & 0 \\
1 & 2 & 2 \\
0 & 2 & 4 \end{pmatrix}\left(\begin{pmatrix}  -\frac{2}{3} \\ 2 \\ 3 \end{pmatrix} +  \alpha_1\begin{pmatrix} 1 \\ -3 \\ 0 \end{pmatrix} \right) \\
& =\frac{15}{2}\alpha^2 - 28\alpha + \frac{100}{3},
\end{align*}
ce qui a lieu en
$$
\alpha_1 = \frac{28}{15},
$$
donnant
$$
x_2 = x_1 + \frac{28}{15}d_1 =
    \begin{pmatrix}
     \frac{6}{5} \\ \frac{-18}{5} \\ 3
    \end{pmatrix}.
$$

La nouvelle longueur de pas est

In [ ]:
α1 = -dot(d1,A*x1)/dot(d1,A*d1)

ou, sous forme de fraction, 28/15:

In [ ]:
28/15

Le nouvel itéré est

In [ ]:
x2 = x1+α1*d1

La recherche linéaire finale à partir de $x_2$ dans la direction $d_2$ requiert de minimiser
$$
f(x_2 + \alpha d_2) = 20 \alpha^2 - 24\alpha + \frac{36}{5},
$$
ce qui a lieu en
$$
\alpha_2 = \frac{3}{5},
$$
donnant
$$
x_3 = x_2 + \frac{3}{5}d_2 =
    \begin{pmatrix}
     0 \\ 0 \\ 0
    \end{pmatrix},
$$
ce qui est bien entendu correct.

Similairement, nous pouvons calculer le nouveau point comme

In [ ]:
α2 = -dot(d2,A*x2)/dot(d2,A*d2)
x3 = x2+α2*d2

$x_3$ est clairement optimal comme $f(x_3) = 0$ et 0 est une borne inférieure sur $f(\cdot)$. Nous pouvons également vérifier que $x_3$ est un point critique. En effet

In [ ]:
∇f(x3)

## Une implémentation naïve

Une première version de l'algorithme du gradient conjugué suit.

In [ ]:
function cg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, trace:: Bool = false)
    n = length(x0)
    x = x0
    g = b+A*x
    d = -g
    if (trace)
        iter = [ x ]
        iterg = [ norm(g) ]
        iterd = [ norm(d) ]
    end
    k = 0
    
    for k = 1:n-1
        Ad = A*d
        normd = dot(d,Ad)
        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = [ iter; [x] ]
            iterg = [ iterg; norm(g)]
            iterd = [ iterd; norm(d) ]
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
    end

    normd = dot(d,A*d)
    α = -dot(d,g)/normd
    x += α*d
    if (trace)
        g = b+A*x # g must be equal to 0
        iter = [ iter; [x] ]
        iterg = [ iterg; norm(g)]
        iterd = [ iterd; norm(d) ]
        return x, iter, iterg, iterd
    end
    
    return x
end

Considérons l'exemple simple

In [ ]:
A = [2 1; 1 2]
b = [1, 0]
A\(-b)

Nous voulons résoudre
$$
    \min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx+c
$$

Ou, de manière équivalente, nous résolvons
$$
    c+\min_{\alpha} f(x) = \frac{1}{2}x^TAx+b^Tx
$$

Appliquons l'algorithme que nous avons implémenté.

In [ ]:
cg_quadratic(A, b, [0, 0], true)

Que se passe-t-il si $A$ n'est pas définie positive? Considérons l'exemple simple suivant:

In [ ]:
A = [ 1 2 ; 2 1]
A\(-b)

In [ ]:
cg_quadratic(A, b, [0, 0], true)

In [ ]:
det(A)

In [ ]:
eigen(A)

In [ ]:
cg_quadratic(A, b, [1, 1], true)

La solution est $x^* = (1/3, -2/3)$. $f(x^*)$ vaut

In [ ]:
f([1/3,-2/3])

Le gradient conjugué trouve la solution du système linéaire, laquelle correspond à un point critique au premier ordre de la fonction.

In [ ]:
∇f = x -> A*x+b

In [ ]:
x = [1.0/3; -2.0/3]
∇f(x)

Mais ce n'est pas un minimum de la fonction! Nous pouvons en effet aisément partir de ce point et diminuer la valeur de la fonction. Construisons la fonction de calcul d'un itéré le long de la plus forte pente.

In [ ]:
step= x -> x-α*∇f(x)

Nous pouvons obtenir une direction de courbure négative en prenant le vecteur propre associé à une valeur propre négative.

In [ ]:
λ, u = eigen(A)

In [ ]:
x = u[:,1] # premier vecteur propre associé à λ = -1
A*x

Remarquons que la norme du vecteur propre ainsi obtenu vaut 1.

In [ ]:
1.0-norm(x)

Construison un point le long de cette direction.

In [ ]:
α = 10
f = x -> 0.5*dot(x,A*x)+dot(b,x)
f(step(x))

La valeur de la fonction en ce point est clairement plus basse que la solution construite par l'algorithme du gradient conjugué!

In [ ]:
x = [1/3.0; -2/3]
f(x)

Remarquons que l'algorithme du gradient conjugué est incapable de réduire la fonction en partant de la solution préalablement trouvée.

In [ ]:
cg_quadratic(A, b, x, true)

Nous devons incorporer un test sur $\nabla f(x_k)$!

Un exemple plus complexe.

In [ ]:
n = 500;
m = 600;
A = randn(n,m);
A = A * A';  # A is now a positive semi-definite matrix
A = A+I # A is positive definite
b = zeros(n)
for i = 1:n
  b[i] = randn()
end
x0 = zeros(n)

In [ ]:
b1 = A\(-b)

In [ ]:
b2, iter, iterg, iterd = cg_quadratic(A, b, x0, true);

À nouveau, nous avons obtenu une solution au système $Ax = b$.

In [ ]:
norm(b1-b2)

Affichons l'historique des normes du gradient.

In [ ]:
iterg

Affichons à présent l'historique des normes de la direction de recherche.

In [ ]:
iterd

Cela fonctionne, mais devons-nous vraiment faire 500 itérations? Nous serions satisfaits si nous sommes proches de la solution. Nous pouvons mesurer le résidu du système linéaire residual of the linear system
$$
r = b+Ax,
$$
ce qui n'est rien d'autre que le gradient de la fonction objectif du problème de minimisation quadratique.

Nous devons inclure un test de convergence dans la fonction.

In [ ]:
function cg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector, trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    d = -g
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    while ((dot(g,g) > tol2) && (k < n))
        Ad = A*d
        normd = dot(d,Ad)
        α = dot(g,g)/normd
#        α = -dot(d,g)/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g = b+A*x
        β = dot(g,Ad)/normd
        d = -g+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

Le nombre d'itérations est à présent

In [ ]:
k

Sommes-nous proche de la solution?

In [ ]:
size(A)

ce qui est nettement moindre que la dimension du problème.

In [ ]:
norm(b1-x)

## Gradient conjugué préconditionné

Si le nombre de conditionnement est égal à 1, nous convergeons en une itération.

Rappelons que le nombre de conditionnement d'une matrice $A$ définie positive est donné par
$$
\kappa(A) = \frac{\lambda_{\max}}{\lambda_{\min}}.
$$
$\kappa(A) = 1$ ssi $A = \gamma I$. Dans ce cas
$$
A = \begin{pmatrix} \gamma & 0 & \cdots & 0 \\ 0 & \gamma & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & \gamma \end{pmatrix}.
$$
Observons que $\lambda_{\max} = \lambda_{\min} = \gamma$.

Le problème quadratique devient alors
$$
f(x) = \frac{1}{2}\gamma x^Tx + b^Tx.
$$

Son gradient est
$$
\nabla f(x) = \gamma x + b.
$$
Il s'annule si
$$
x = -\frac{b}{\gamma}.
$$

Soit $x_0$. L'algorithme du gradient conjugué donne comme première de recherche $d_0 = -\nabla f(x_0) = -\gamma x_0 - b$.

Nous avons aussi
$$
\alpha_0 = - \frac{d_0^T\nabla f(x_0)}{d_0^TAd_0} = \frac{\| d_0 \|^2}{\gamma \| d_0 \|^2} = \frac{1}{\gamma}.
$$

Le premier itéré donne
$$
x_1 = x_0 + \alpha_0 d_0 = x_0 + \frac{1}{\gamma} (-\gamma x_0 - b) = -\frac{b}{\gamma}.
$$
ce qui correspond bien à la solution!

Si la matrice $A$ est diagonale et tous les éléments de la diagonale sont identiques, la direction de plus forte pente donne le minimum global.

Une implémentation basique d'un algorithme de gradient préconditionné suit, où $M$ est l'inverse du préconditioneur à appliquer.

In [ ]:
function pcg_quadratic_tol(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix,
                           trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    v = M*g
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k < n))
#    while ((dot(g,g) > tol2) && (k < n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g += α*Ad
        v = M*g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

Vérifions tout d'abord qu'en l'absence de préconditionnement, nous obtenons les mêmes itérés.

Posons

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

In [ ]:
k, norm(x-b1)

Nous pouvons calculer les valeurs propres et le nombre de conditionnement de la matrice $A$.

In [ ]:
eigen(A)

In [ ]:
cond(A)

In [ ]:
A

Essayons de construire un préconditionneur simple en utilisant l'inverse de la diagonale de la matrice $A$.

In [ ]:
D = 1 ./diag(A)
M = Diagonal(D)

Malheureusement, dans ce cas, nous n'observons pas une amélioration du nombre de conditionnement.

In [ ]:
B = M*A
cond(B)

Considérons une autre situation où $A$ est diagonale.

In [ ]:
n = 1000;
A = zeros(n,n);
for i = 1:n
    A[i,i] = 10*rand()
end
b = zeros(n)
for i = 1:n
  b[i] = rand()
end
x0 = zeros(n)
cond(A)

La solution que nous cherchons est

In [ ]:
A\b

Sans préconditionnement, nous avons la séquence d'itérés

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Ceci est équivalent à la version de l'algorithme non-préconditionné.

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

Cependant, puisque $A$ est diagonal, un préconditionneur diagonal évident est $A^{-1}$ lui-même.

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

Le nombre de préconditionnement de la matrice préconditionnée est bien entendu égal à 1.

In [ ]:
cond(M*A)

La théorie prédit alors que nous convergeons en une itération avec le gradient conjugué précontionné.

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Mais le cas diagonal n'est pas très intéressant, comme le système peut être résolut facilement, en traitant les variables indépendamment.

Considérons à présent un autre exemple.

In [ ]:
A = zeros(n,n)+3*I
for i = 1:n-1
    A[i,i+1] = 1.4
    A[i+1,i] = 1.4
end
A

Les valeurs propres de la matrice sont

In [ ]:
eigen(A)

La matrice est donc bien définie positive et la solution du système est

In [ ]:
A\(-b)

L'algorithme du gradient conjugué donne

In [ ]:
x, iter, k = cg_quadratic_tol(A, b, x0, true)

Essayons à nouveau comme précondtionneur l'inverse de la diagonale.

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end

Comparons les nombres de conditionnement de la matrice avec et sans conditionnement.

In [ ]:
cond(A), cond(M*A)

Le conditionnement de la matrice n'a pas changé, aussi nous ne devrions pas voir de différence lors de l'application de l'algorithme du gradient conjugué.

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

De fait, il n'y a pas d'avantage notable. Notons de plus la structure de $A^{-1}$.

In [ ]:
M = inv(A)

Peut-on exploiter la structure creuse des matrices? La librairie `SparseArrays` vient ici à notre secours.

In [ ]:
using SparseArrays

sparse(A)

In [ ]:
sparse(M)

Nous pouvons visualier que l'inversion transforme une matrice creuse en matrice dense. Ce n'est pas intéressant, même si, évidemment, nous n'avons à présent besoin que d'une itération avec l'algorithme du gradient conjugé.

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Considérons l'exemple suivant.

In [ ]:
n = 1000
A = zeros(n,n)+Diagonal([2+i*i for i=1:n])

for i = 1:n-1
    A[i,i+1] = 1
    A[i+1,i] = 1
end
A[n,1] = 1
A[1,n] = 1
A

La matrice ainsi construite est particulièrement creuse. De telles structures apparaissent des divers problèmes physiques.

Nous l'avons construite de telle manière à avoir un très mauvais conditionnement.

In [ ]:
cond(A)

Le taux de convergence prédit est très lent.

In [ ]:
κ = cond(A)
(sqrt(κ)-1)/(sqrt(κ)+1)

Mais à nouveau, l'inverse de la matrice est dense.

In [ ]:
A^(-1)

Essayons tout d'abord avec la matrice identité (autrement dit, nous n'appliquons pas de préconditionnement).

In [ ]:
M = zeros(n,n)+I
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Le nombre maximum d'itérations est atteint, pour une précision restant moyenne. En effet, si nous comparons avec la solution recherchée, nous obtenons

In [ ]:
xopt = A\(-b)
x-xopt

Essayons à nouveau de préconditionner le système avec l'inverse de la diagonale.

In [ ]:
M = zeros(n,n)
for i = 1:n
    M[i,i] = 1/A[i,i]
end
cond(A*M), cond(A)

Le conditionnement de la matrice préconditionnée est cette fois-ci nettement meilleur, suggérant que $M$ est un bon préconditionneur. Essayons done l'algorithme du gradient conjugué préconditionné.

In [ ]:
x, iter, k = pcg_quadratic_tol(A, b, x0, M, true)

Non seulement, le nombre d'itérations est nettement inférieur, mais la précision est grandement améliorée:

In [ ]:
x-xopt

Le préconditionnement est cependant souvent utilisé pour produire une matrice proche de $A$, mais permettant de résoudre de manière plus efficace des systèmes linéaires. La version suivante de l'algorithme du gradient conjugué préconditionné.

Deux versions sont proposées, une avec une matrice régulière, une avec une matrice triangulaire inférieure, sachant que résoudre un système triangulaire est direct.

In [ ]:
function pcg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, M:: Matrix,
                       trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    v = M\g    # l'application de M se fait à présent en calculant un système linéaire plutôt qu'une multiplication matricielle.
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g += α*Ad
        v = M\g
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

function pcg_quadratic(A:: Matrix, b:: Vector, x0:: Vector, L:: LowerTriangular,
                       trace:: Bool = false, tol = 1e-8)
    n = length(x0)
    x = x0
    if (trace)
        iter = [ x ]
    end
    g = b+A*x
    U = transpose(L)
    v = U\(L\g)    # l'application de M se fait à présent en calculant un système linéaire plutôt qu'une multiplication matricielle.
    d = -v
    k = 0
    
    tol2 = tol*tol

    β = 0.0

    gv = dot(g,v)
    while ((gv > tol2) && (k <= n))
#    while ((dot(g,g) > tol2) && (k <= n))
        Ad = A*d
        normd = dot(d,Ad)
        #gv = dot(g,v)
        α = gv/normd
        x += α*d
        if (trace)
            iter = [ iter; x ]
        end
        g += α*Ad
        v = U\(L\g)
        gvold = gv
        gv = dot(g,v)
        β = gv/gvold
        d = -v+β*d
        k += 1
    end

    if (trace)
        iter = [ iter; x ]
        return x, iter, k
    end

    return x, k
end

Un préconditionneur populaire est la factorisation de Cholesky incomplète, qui reproduit la factorisation de Cholesky, mais en ne considérant que les positions où l'élément de la matrice initiale est non nul, avec de conserver le caractère creux (si présent), et en s'arrêtant prématurément si la matrice se révèle non définie positive.

In [ ]:
function ichol(A:: Matrix)

    n = size(A,1)
    C = LowerTriangular(zeros(n,n)+I)
    
    for k=1:n
        C[k,k] = sqrt(A[k,k])
        for i=(k+1):n
            if (A[i,k] != 0)
                C[i,k] = A[i,k]/A[k,k]    
            end
        end
        for j=(k+1):n
            for i=j:n
                if (A[i,j] != 0)
                    C[i,j] = A[i,j]-A[i,k]*A[j,k]
                end
            end
        end
    end

    return C
end

Essayons d'abord avec la factorisation complète de A.

In [ ]:
C = cholesky(A)
C.L

In [ ]:
M = C.L*C.U

Si nous essayons d'appliquer l'algorithme du gradient conjugué préconditionné, nous voyons que nous convergeons en une itération, ce qui est logique vu que $M = A$.

In [ ]:
x, iter, k = pcg_quadratic(A, b, x0, M, true)

Néanmoins, l'algorithme converge plus rapidement en utilisant le facteur de Cholesky directement plutôt que $M$.

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark pcg_quadratic(A, b, x0, M, true)

In [ ]:
@benchmark pcg_quadratic(A, b, x0, C.L, true)

Travailler avec le facteur de Cholesky accélère grandement la résolution du système!

Travaillons à présent avec la factorisation incomplète.

In [ ]:
C = ichol(A)

La factorisation reste incomplète, comme illustré ci-dessous.

In [ ]:
M=C*C'
norm(M-A)

Nous pouvons néanmoins utiliser ce facteur pour résoudre le système.

In [ ]:
x, iter, k = pcg_quadratic(A, b, x0, M, true)

Nous prenons à présent 6 itérations, comme la factorisation n'a pas été conduite jusqu'au bout.

In [ ]:
@benchmark pcg_quadratic(A, b, x0, M, true)

In [ ]:
@benchmark pcg_quadratic(A, b, x0, C, true)

Travailler avec la matrice sous forme plein ralentit le code en raison du nombre plus grand d'itérations, mais la version triangulaire est encore plus rapide.

Mais dans tous les cas, nous sommes bien plus efficace que la version non préconditionnée.

In [ ]:
M = zeros(n,n)+I
@benchmark pcg_quadratic(A, b, x0, M, true)

Notons aussi que la version proposée du gradient conjuguée, avec le préconditioneur triangulaire, est plus rapide que la résolution directe.

In [ ]:
@benchmark A\(-b)

Évidemment, ceci néglige le temps utlisé pour calculer le facteur. Malheureusement, le code implémenté n'est pas encore très efficace...

In [ ]:
@benchmark ichol(A)

Une implémentation efficace ferait usage des matrices creuses et de fonctions spécifiques pour caluler $v$.